# Linear reservoir - Bivariate calibration with SCE-UA
***

**Autor:** Chus Casado<br>
**Date:** 13-06-2024<br>

**To do:**<br>

**Questions:**<br>

In [6]:
import sys
sys.path.append('../../src/')
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import spotpy
from spotpy.objectivefunctions import kge
import yaml
from pathlib import Path
from tqdm.notebook import tqdm
import glob
# import cartopy.crs as ccrs
# import cartopy.feature as cf

# from spot_setup_lisflood import spot_setup3, spot_setup5
from lisfloodreservoirs.calibration.bivariate_linear import bivariate
from lisfloodreservoirs.reservoirs.linear import Linear

In [2]:
from lisfloodreservoirs.utils.metrics import KGEmod

## Configuration

In [3]:
with open('config_linear_2var.yml', 'r', encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

### Paths

PATH_DATASET = Path(cfg['paths']['dataset'])

### Reservoir model

MODEL = cfg['model'].lower()

### Calibration

# # sequential mode
# parallel = "seq"  

# calibration parameters
ALGORITHM = cfg['calibration']['algorithm'].lower()
TARGET = cfg['calibration']['target']
MAX_ITER = cfg['calibration'].get('max_iter', 1000)
COMPLEXES = cfg['calibration'].get('COMPLEXES', 4)
TRAIN_SIZE = cfg['calibration'].get('TRAIN_SIZE', 0.7)

# results will be saved in this path
PATH_OUT = Path(cfg['calibration']['path_out'])
PATH_OUT = PATH_OUT / MODEL / ALGORITHM
if len(TARGET) == 1:
    TARGET = TARGET[0]
    PATH_OUT = PATH_OUT / 'univariate' / TARGET
elif len(TARGET) == 2:
    PATH_OUT /= 'bivariate'
else:
    print('ERROR. Only univariate or bivariate calibrations are supported')
    exit
PATH_OUT.mkdir(parents=True, exist_ok=True)
print(f'Results will be saved in {PATH_OUT}')

Results will be saved in calibration\linear\sceua\bivariate


## Data

### GloFAS

#### Reservoirs

In [4]:
# load shapefile of GloFAS reservoirs
reservoirs = gpd.read_file('../../GIS/reservoirs_analysis_US.shp')
reservoirs.set_index('ResID', drop=True, inplace=True)

print(f'{reservoirs.shape[0]} reservoirs in the shape file')

94 reservoirs in the shape file


#### Time series

In [5]:
# read GloFAS time series
path = Path('../../data/reservoirs/GloFAS/long_run')
glofas_ts = {}
for file in tqdm(glob.glob(f'{path}/*.csv')):
    id = int(file.split('\\')[-1].split('.')[0].lstrip('0'))
    if id not in reservoirs.index:
        continue
    glofas_ts[id] = pd.read_csv(file, parse_dates=True, dayfirst=False, index_col='date')
    
print(f'{len(glofas_ts)} reservoirs in the GloFAS time series')

# convert storage time series into volume
for id, df in glofas_ts.items():
    df.storage *= reservoirs.loc[id, 'CAP'] * 1e6

# period of GloFAS simulation
start, end = glofas_ts[id].first_valid_index(), glofas_ts[id].last_valid_index()

  0%|          | 0/143 [00:00<?, ?it/s]

94 reservoirs in the GloFAS time series


### ResOpsUS
#### Time series

In [7]:
resops_ts = {}
for glofas_id in tqdm(reservoirs.index):
    # load timeseries
    grand_id = reservoirs.loc[glofas_id, 'GRAND_ID']
    series_id = pd.read_csv(PATH_DATASET / 'time_series_all' / f'ResOpsUS_{grand_id}.csv', parse_dates=True, index_col='date')
    # remove empty time series
    series_id = series_id.loc[start:end]#.dropna(axis=1, how='all')
    # remove duplicated index
    series_id = series_id[~series_id.index.duplicated(keep='first')]
    # save in dictionary
    resops_ts[glofas_id] = series_id

print(f'{len(resops_ts)} reservoirs in the ResOpsUS time series')
    
# convert storage from hm3 to m3
for id, df in resops_ts.items():
    df.storage *= 1e6

  0%|          | 0/94 [00:00<?, ?it/s]

94 reservoirs in the ResOpsUS time series


## Calibration

In [ ]:
for ResID in tqdm(reservoirs.index):
    
    # file where the calibration results will be saved
    dbname = f'{PATH_OUT}/{ResID:03}_samples'
    if os.path.isfile(dbname + '.csv'):
        print(f'The file {dbname}.csv already exists.')
        continue   

    ## TIME SERIES
    try:
        # observed time series
        obs = resops_ts[ResID][['storage', 'inflow', 'outflow']].copy()
        obs[obs < 0] = np.nan

        # define calibration period
        if obs.outflow.isnull().all():
            print(f'Reservoir {ResID} is missing outflow records')
            continue
        elif obs.storage.isnull().all():
            print(f'Reservoir {ResID} is missing storage records')
            continue
        else:
            start_obs = max([obs[var].first_valid_index() for var in ['storage', 'outflow']])
            end_obs = min([obs[var].last_valid_index() for var in ['storage', 'outflow']])
            cal_days = timedelta(days=np.floor((end_obs - start_obs).days * TRAIN_SIZE))
            start_cal = end_obs - cal_days

        # define train and test time series
        x_train = glofas_ts[ResID].inflow[start_cal:end_obs]
        y_train = obs.loc[start_cal:end_obs, ['storage', 'outflow']]
        x_test = glofas_ts[ResID].inflow[start:start_cal]
        y_test = obs.loc[start_obs:start_cal, ['storage', 'outflow']]
        
    except Exception as e:
        print(f'ERROR. The time series of reservoir {ResID} could not be set up\n', e)
        continue

    ## SET UP SPOTPY
    try:
        # extract GloFAS reservoir parameters
        Vmin, Vtot, Qmin = reservoirs.loc[ResID, ['clim', 'CAP', 'minq']]
        Vtot *= 1e6
        Vmin *= Vtot

        # initialize the calibration setup of the LISFLOOD reservoir routine
        setup = bivariate(inflow=x_train, 
                           storage=y_train.storage, 
                           outflow=y_train.outflow,
                           Vmin=Vmin, 
                           Vtot=Vtot, 
                           Qmin=Qmin,
                           obj_func=KGEmod)

        # define the sampling method
        sceua = spotpy.algorithms.sceua(setup, dbname=dbname, dbformat='csv', save_sim=False)
    except Exception as e:
        print(f'ERROR. The SpotPY set up of reservoir {ResID} could not be done\n', e)
        continue
        
    ## LAUNCH SAMPLING
    try:
        # start the sampler
        sceua.sample(MAX_ITER, ngs=COMPLEXES, kstop=3, pcento=0.01, peps=0.1)
    except Exception as e:
        print(f'ERROR. While sampling the reservoir {ResID}\n', e)
        continue

    ### VALIDATION
    # read CSV of results
    try:
        results = pd.read_csv(f'{dbname}.csv')
        results.index.name = 'iteration'
        parcols = [col for col in results.columns if col.startswith('par')]
    except Exception as e:
        print(f'ERROR while reading results form reservoir {ResID}\n', e)
        continue
    
    # compute validation KGE of each simulation and overwrite CSV file
    try:       
        results['KGEval'] = np.nan
        for i in tqdm(results.index):
            Q_sim, V_sim = setup.simulation(pars=results.loc[i, parcols], 
                                            inflow=x_test, 
                                            storage_init=y_test.storage[0])
            results.loc[i, 'KGEval'] = 1 - np.sqrt(np.sum([(1 - KGEmod(y_test.outflow, Q_sim)[0])**2, (1 - KGEmod(y_test.storage, V_sim)[0])**2]))
        results.to_csv(f'{dbname}.csv', index=False, float_format='%.8f')
    except Exception as e:
        print(f'ERROR while computing KGE for the validation period in reservoir {ResID}\n', e)
    
    # select optimal parameters (best validation) and export them
    try:
        best_iter = results.KGEval.idxmax() # results.like1.idxmin()
        parvalues = {col[3:]: float(results.loc[best_iter, col]) for col in parcols}
        with open(f'{PATH_OUT}/{ResID:03}_optimal_parameters.yml', 'w') as file:
            yaml.dump(parvalues, file)
    except Exception as e:
        print(f'ERROR while searching for optimal parameters in reservoir {ResID}\n', e)
        continue
    
    # simulate the whole observed period with the optimal parameterization
    try:       
        res = Linear(Vmin, Vtot, Qmin, T=parvalues['T'])
        sim = res.simulate(glofas_ts[ResID].inflow[start_obs:end_obs],
                           obs.storage[start_obs])
        res.scatter(sim,
                    obs,
                    norm=False,
                    title=ResID,
                    save=PATH_OUT / f'{ResID:03}_scatter.jpg'
                   )
        res.lineplot({'GloFAS': glofas_ts[ResID], 'cal': sim},
                     obs,
                     save=PATH_OUT / f'{ResID:03}_lineplot.jpg'
                    )
    except Exception as e:
        print(f'ERROR while simulating with optimal parameters in reservoir {ResID}\n', e)

  0%|          | 0/94 [00:00<?, ?it/s]

Initializing the  Shuffled Complex Evolution (SCE-UA) algorithm  with  1000  repetitions
The objective function will be minimized
Starting burn-in sampling...


  0%|          | 0/9714 [00:00<?, ?it/s]

Initialize database...
['csv', 'hdf5', 'ram', 'sql', 'custom', 'noData']
* Database file 'calibration\linear\sceua\bivariate/007_samples.csv' created.


  0%|          | 0/9714 [00:00<?, ?it/s]

...

C:\DEV\Anaconda3\envs\xr\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\DEV\Anaconda3\envs\xr\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


  0%|          | 0/5287 [00:00<?, ?it/s]

  0%|          | 0/5287 [00:00<?, ?it/s]

  0%|          | 0/5287 [00:00<?, ?it/s]

  0%|          | 0/11753 [00:00<?, ?it/s]